In [1]:
pip uninstall autogen-agentchat pyautogen -y

Found existing installation: autogen-agentchat 0.4.9.3
Uninstalling autogen-agentchat-0.4.9.3:
  Successfully uninstalled autogen-agentchat-0.4.9.3
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install autogen-agentchat

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show autogen-agentchat

Name: autogen-agentchat
Version: 0.4.9.3
Summary: AutoGen agents and teams library
Home-page: 
Author: 
Author-email: 
License: MIT License
        
            Copyright (c) Microsoft Corporation.
        
            Permission is hereby granted, free of charge, to any person obtaining a copy
            of this software and associated documentation files (the "Software"), to deal
            in the Software without restriction, including without limitation the rights
            to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
            copies of the Software, and to permit persons to whom the Software is
            furnished to do so, subject to the following conditions:
        
            The above copyright notice and this permission notice shall be included in all
            copies or substantial portions of the Software.
        
            THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
            IMPLIED, INCLUDING B

In [16]:
import os
import json
import asyncio
import re
import datetime
from typing import Sequence, Union, Optional

# --- Gerekli Kütüphaneleri İçe Aktarma (GÜNCELLENMİŞ YOLLAR) ---
# import from autogen yok! Doğrudan alt modüllerden import ediyoruz.


# Ajanlar, Takımlar, Mesajlar vb. (autogen_agentchat altında)
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import (
    AgentEvent, ChatMessage, TextMessage, ToolCallRequestEvent,
    ToolCallExecutionEvent, ToolCallSummaryMessage
)

# LLM İstemcisi (autogen_ext altında)
from autogen_ext.models.openai import OpenAIChatCompletionClient

# UI (opsiyonel, konsol çıktısı için) (autogen_agentchat altında)
from autogen_agentchat.ui import Console

# Diğer gerekli kütüphaneler
from duckduckgo_search import DDGS

# --- Yer Tutucu Fonksiyonlar (Değişiklik Yok) ---

def log_event(event_details: str) -> str:
    log_file = "assistant_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] {event_details}\n")
        return f"Event logged successfully to {log_file}: {event_details}"
    except Exception as e:
        print(f"Error logging event: {e}")
        return f"Error logging event: {e}"

def perform_web_search(query: str, max_results: int = 3) -> str:
    print(f"[Web Arama] DuckDuckGo ile aranıyor: '{query}' (Maks {max_results} sonuç)")
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=max_results))
            if not results:
                return f"No search results found for '{query}'."
            formatted_results = "\n\n".join([f"Title: {res.get('title', 'N/A')}\nLink: {res.get('href', 'N/A')}\nSnippet: {res.get('body', 'N/A')}" for res in results])
            return f"Search results for '{query}':\n{formatted_results}"
    except Exception as e:
        print(f"[Hata] Web araması sırasında hata oluştu: {e}")
        return f"Error during web search for '{query}': {e}"

def save_email_draft(to: str, subject: str, body: str) -> str:
    draft_file = "email_drafts.txt"
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    draft_content = f"""--- Draft Saved: {timestamp} ---
To: {to}
Subject: {subject}
--- Body ---
{body}
--- End Draft ---
"""
    print(f"[E-posta Taslağı] Taslak '{draft_file}' dosyasına kaydediliyor...")
    try:
        with open(draft_file, "a", encoding="utf-8") as f:
            f.write(draft_content + "\n\n")
        return f"Email draft for {to} with subject '{subject}' saved to {draft_file}."
    except Exception as e:
        print(f"[Hata] E-posta taslağı kaydedilirken hata oluştu: {e}")
        return f"Error saving email draft: {e}"

def add_todo_item(task: str) -> str:
    todo_file = "todo_list.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    print(f"[Yapılacaklar Listesi] Görev '{todo_file}' dosyasına ekleniyor: '{task}'")
    try:
        with open(todo_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] - {task}\n")
        return f"Task '{task}' added to the To-Do list ({todo_file})."
    except Exception as e:
        print(f"[Hata] Yapılacaklar listesine eklenirken hata oluştu: {e}")
        return f"Error adding task to To-Do list: {e}"

# --- Yapılandırma (LLM İstemcisi Oluşturma - config_list_from_json OLMADAN) ---

config_list_path = "OAI_CONFIG_LIST.json" # JSON dosyanızın yolu
try:
    # JSON dosyasını manuel olarak oku
    with open(config_list_path, 'r', encoding='utf-8') as f:
        config_data = json.load(f)

    # Genellikle JSON bir liste içerir, ilk öğeyi alalım
    if isinstance(config_data, list) and len(config_data) > 0:
        llm_config_params = config_data[0]
    elif isinstance(config_data, dict): # Eğer liste değilse, doğrudan sözlük olabilir
         llm_config_params = config_data
    else:
        raise ValueError(f"{config_list_path} dosyasında geçerli yapılandırma bulunamadı.")

    # Gerekli parametreleri al
    # API anahtarı için önce ortam değişkenini kontrol et, sonra JSON'u
    api_key = os.getenv("OPENAI_API_KEY", llm_config_params.get("api_key"))
    model_name = llm_config_params.get("model", "gpt-4o") # Varsayılan model veya JSON'dan gelen

    if not api_key:
        raise ValueError("OpenAI API anahtarı ne JSON dosyasında ne de OPENAI_API_KEY ortam değişkeninde bulunamadı.")

    # OpenAIChatCompletionClient istemcisini oluştur
    model_client = OpenAIChatCompletionClient(
        model=model_name,
        api_key=api_key,
        # Gerekirse diğer parametreleri ekleyin (örn. base_url, api_version llm_config_params'dan alınabilir)
        # temperature=0.5 # Sıcaklık gibi ayarlar run metodunda verilebilir
    )
    print(f"OpenAI İstemcisi '{model_name}' modeli için başarıyla oluşturuldu.")

except FileNotFoundError:
    print(f"Hata: Yapılandırma dosyası bulunamadı: {config_list_path}")
    exit()
except (json.JSONDecodeError, ValueError, KeyError) as e: # KeyError eklendi
    print(f"Hata: Yapılandırma hatası ({config_list_path}): {e}")
    exit()
except Exception as e:
    print(f"Beklenmeyen bir hata oluştu: {e}")
    exit()


# --- Ajan Tanımları (model_client ve description ile) ---
# Ajan tanımları önceki güncellenmiş haliyle aynı kalabilir, sadece importlar değişti.

# Orkestratör / Yönetici Ajan
orchestrator = AssistantAgent(
    name="Orchestrator",
    model_client=model_client,
    system_message="""You are the Orchestrator/Supervisor Agent... (önceki sistem mesajı)... End the entire process with "TERMINATE" when the user request is fully addressed.""",
    description="Manages the overall workflow, analyzes user requests, delegates tasks to specialized agents, and ensures the task completion."
)

# Kullanıcı Proxy Ajanı
user_proxy = UserProxyAgent(
    name="User_Proxy",
    description="Represents the end-user. Initiates requests and receives the final response."
)

# Girdi Ön İşleme Ajanı
input_preprocessor = AssistantAgent(
    name="Input_Preprocessor",
    model_client=model_client,
    system_message="""You are the Input Preprocessing Agent... Report only with 'Cleaned Query: [the cleaned query text].'""",
    description="Cleans and preprocesses the initial user query."
)

# Niyet ve Görev Yönlendirme Ajanı
intent_task_router = AssistantAgent(
    name="Intent_Task_Router",
    model_client=model_client,
    system_message="""You are the Intent & Task Routing Agent... Report the plan back to the Orchestrator. Example Response: 'Plan: 1. Web_Search_Agent (query='...'), ...'""",
    description="Analyzes the query to determine intent and creates an execution plan."
)

# --- Uzmanlaşmış Görev Ajanları (Tools ile) ---

# Web Arama Ajanı
web_search_agent = AssistantAgent(
    name="Web_Search_Agent",
    model_client=model_client,
    tools=[perform_web_search],
    reflect_on_tool_use=True,
    system_message="""You are the Web Search Agent... Use the 'perform_web_search' tool... Report results.""",
    description="Performs web searches using the 'perform_web_search' tool."
)

# Özetleme Ajanı
summarization_agent = AssistantAgent(
    name="Summarization_Agent",
    model_client=model_client,
    system_message="""You are the Summarization Agent... Summarize text... Provide summary.""",
    description="Summarizes provided text based on instructions."
)

# Zamanlama ve Takvim Ajanı (Simülasyonlu)
scheduling_calendar_agent = AssistantAgent(
    name="Scheduling_Calendar_Agent",
    model_client=model_client,
    system_message="""You are the Scheduling & Calendar Agent... Simulate operation... Report simulated outcome.""",
    description="Simulates scheduling and calendar operations."
)

# E-posta Oluşturma ve Duygu Analizi Ajanı
email_sentiment_agent = AssistantAgent(
    name="Email_Composition_Sentiment_Agent",
    model_client=model_client,
    tools=[save_email_draft],
    reflect_on_tool_use=True,
    system_message="""You are the Email Composition & Sentiment Analysis Agent... Use 'save_email_draft' tool... Report result.""",
    description="Drafts emails using 'save_email_draft' tool or analyzes sentiment."
)

# Yapılacaklar Listesi Ajanı
todo_list_agent = AssistantAgent(
    name="To_Do_List_Agent",
    model_client=model_client,
    tools=[add_todo_item],
    reflect_on_tool_use=True,
    system_message="""You are the To-Do List Agent... Use 'add_todo_item' tool... Report result.""",
    description="Adds tasks to a to-do list using the 'add_todo_item' tool."
)

# --- Destekleyici Ajanlar ---

# Takip ve Açıklama Ajanı
follow_up_clarification_agent = AssistantAgent(
    name="Follow_Up_Clarification_Agent",
    model_client=model_client,
    system_message="""You are the Follow-Up & Clarification Agent... Formulate a question... Report *only the question*.""",
    description="Formulates clarifying questions for the user."
)

# Analiz ve Loglama Ajanı
analytics_logging_agent = AssistantAgent(
    name="Analytics_Logging_Agent",
    model_client=model_client,
    tools=[log_event],
    reflect_on_tool_use=True,
    system_message="""You are the Analytics & Logging Agent... Use the 'log_event' tool... Report success.""",
    description="Logs specific events using the 'log_event' tool."
)

# --- Grup Sohbeti Kurulumu (SelectorGroupChat ile) ---

agents = [
    user_proxy, orchestrator, input_preprocessor, intent_task_router,
    web_search_agent, summarization_agent, scheduling_calendar_agent,
    email_sentiment_agent, todo_list_agent, follow_up_clarification_agent,
    analytics_logging_agent,
]

# Sonlandırma Koşulları
text_mention_termination = TextMentionTermination(
    text="TERMINATE",
    sources=[orchestrator.name]
)
max_messages_termination = MaxMessageTermination(max_messages=50)

# İki koşulu OR (|) operatörü ile birleştir (GÜNCELLENDİ)
combined_termination_condition = text_mention_termination | max_messages_termination

selector_prompt = """You are overseeing a conversation between agents.
Available roles:
{roles}

Conversation history:
{history}

The participants are: {participants}.
The Orchestrator agent is responsible for managing the workflow and delegating tasks.
Read the conversation history carefully.
Based on the last messages and the overall goal, select the *next* agent to speak.
*   If the Orchestrator just gave a clear instruction to a specific agent, select that agent.
*   If an agent just finished its task and reported back, the Orchestrator should probably speak next to process the result or delegate the next step.
*   If the conversation seems stuck or needs direction, the Orchestrator should speak.
*   If the user needs clarification, the Follow_Up_Clarification_Agent might be needed (usually instructed by Orchestrator).
Only return the name of the single agent selected to speak next. Do not add any other text.
"""


# SelectorGroupChat Takımını Oluştur (GÜNCELLENDİ)
team = SelectorGroupChat(
    participants=agents,
    model_client=model_client, # LLM bu istemciyi kullanarak seçim yapacak
    termination_condition=combined_termination_condition,
    # selector_func=custom_selector_func, # KALDIRILDI
    selector_prompt=selector_prompt,     # EKLENDİ - LLM'e rehberlik eder
    allow_repeated_speaker=True,       # LLM aynı ajanı (örn. Orkestratör) tekrar seçebilir
)

# --- Sohbeti Başlatma (Asenkron) ---

user_query = (
    "Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, "
    "summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) "
    "explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list."
)

print(f"--- Sohbet Başlatılıyor --- \nSorgu: {user_query}\n{'='*30}")

async def run_team_chat_stream():
    # team.run() yerine team.run_stream() kullanıyoruz
    # Console, akıştaki mesajları otomatik olarak yazdıracak
    stream = team.run_stream(task=user_query)
    final_result = await Console(stream) # Console akışı işler ve son TaskResult'ı döndürür

    print(f"\n{'='*30}\n--- Sohbet Sona Erdi ---")
    # final_result None olabilir eğer stream'de hata olursa, kontrol edelim
    if final_result:
        print(f"Sonlanma Nedeni: {final_result.stop_reason}")
        print(f"Toplam Mesaj Sayısı: {len(final_result.messages)}")
        messages = final_result.messages
    else:
        print("Sohbet sonucu alınamadı.")
        messages = [] # Hata durumunda boş liste


    # --- Son Çıktıyı İşleme (Önceki gibi ama 'messages' listesini kullanır) ---
    final_output = "Nihai çıktı işlenemedi veya bulunamadı."
    log_summary = []

    if messages:
        # Mesaj işleme döngüsü aynı kalır...
        for msg in reversed(messages):
            speaker_name: Optional[str] = None
            speaker_ref = getattr(msg, 'source', None)
            if hasattr(speaker_ref, 'name'):
                speaker_name = speaker_ref.name
            elif isinstance(speaker_ref, str):
                speaker_name = speaker_ref

            content_str = str(getattr(msg, 'content', ''))

            if isinstance(msg, TextMessage) and speaker_name == orchestrator.name:
                 if content_str.strip() and "TERMINATE" not in content_str:
                     final_output = f"(Son Orkestratör mesajı):\n{content_str.strip()}"
                     # break # Logları toplamak için devam et
                 elif content_str.strip() and "TERMINATE" in content_str and final_output.startswith("Nihai çıktı"):
                      final_output = f"(TERMINATE içeren son Orkestratör mesajı):\n{content_str.replace('TERMINATE','').strip()}"

            elif speaker_name == analytics_logging_agent.name:
                 if "logged successfully" in content_str:
                      log_summary.append(content_str)


    print(f"\n--- Nihai Çıktı ---")
    print(final_output)

    print("\n--- Loglanan Olaylar Özeti ---")
    if log_summary:
        for log_entry in reversed(log_summary): # Kronolojik sıra
            print(f"- {log_entry}")
    else:
        print("Loglanan olay bulunamadı.")

    if os.path.exists("assistant_log.txt"):
        print("\nLog detayları 'assistant_log.txt' dosyasına yazıldı.")

# Asenkron sohbeti çalıştır
if __name__ == "__main__":
    try:
        # run_team_chat yerine run_team_chat_stream'i çağırıyoruz
        await run_team_chat_stream() # Jupyter/IPython için await
    except Exception as e:
        print(f"\n--- Çalışma Zamanı Hatası ---")
        import traceback
        traceback.print_exc()

OpenAI İstemcisi 'gpt-4-1106-preview' modeli için başarıyla oluşturuldu.
--- Sohbet Başlatılıyor --- 
Sorgu: Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list.
---------- user ----------
Can you search the web for the main benefits of using the Autogen framework for multi-agent systems, summarize the key points into 3 bullets, and then draft an email to my colleague Alex (alex@example.com) explaining these benefits? Also, add 'Prepare presentation on Autogen' to my to-do list.
---------- Web_Search_Agent ----------
[FunctionCall(id='call_XXcKrTqwBxaO2qKORu8OfQsH', arguments='{"query":"benefits of using Autogen framework for multi-agent systems","max_results":3}', name='perform_web_search')]
---------- Web_Search_Agent ----------
[FunctionExecutio